In [1]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime

def top_to_bottom(browser, i):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(1.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break
                
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
        df['연락처'] = df['연락처'].str.split('\+82').str.get(1).str[0:13].fillna('없음')
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        int_numbers = []
        for i in df['연락처']:
            if "10" in i:
                ii = "0" + i[1:]
                int_numbers.append(ii)
            else:
                int_numbers.append('없음')
        df['연락처'] = int_numbers
        df.to_csv(f"{f_name}_utf-8.csv", encoding='UTF-8')
    except:
        print(len(name)+' '+len(contact)+' '+len(city))
#         df.to_csv(f"{f_name}_cp949.csv", encoding='CP949')


options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

#open browser
browser.get("https://www.facebook.com/")
time.sleep(2)

#log info, search keyword
USER = "shm8485@gmail.com"
PWD = "sohee8485"

#login
elem_id = browser.find_element("id", "email")
pyperclip.copy(USER)
elem_id.send_keys(Keys.CONTROL, "v")
time.sleep(2)

elem_pw = browser.find_element("id", "pass")
pyperclip.copy(PWD)
elem_pw.send_keys(Keys.CONTROL, "v")
elem_pw.send_keys("\n")
time.sleep(3)

#search
keyword = '음식'
kw_city = None
browser.get(f"https://www.facebook.com/search/groups/?q={keyword}")
time.sleep(2)

try:
    elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
    elem_block.click()
except:
    pass
try:
    elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
    elem_block.click()
except:
    pass
time.sleep(1)


#show only public groups
elem_public = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[2]/div/div[2]/div/input')
elem_public.click()

#city
if kw_city:
    elem_city = browser.find_element("xpath", "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div")
    time.sleep(1)
    elem_city.click()
    elem_city = browser.find_element("xpath", '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div/input')
    time.sleep(1)
    pyperclip.copy(kw_city)
    elem_city.send_keys(Keys.CONTROL, "v")
    time.sleep(1)         
    elem_city = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[2]/div/div/div[1]/div[1]/div/ul/li[1]/div/div[1]/div/div/div')
    elem_city.click()
    time.sleep(1) 

# top_to_bottom(browser, 5)
time.sleep(3)

soup_groups = bs(browser.page_source, "html.parser")
top_elements = soup_groups.find_all(class_="x1yztbdb")
groups_href = []

try:
    for i in top_elements:
        groups_href.append(i.find("a").attrs['href'])
except:
    for i in top_elements:
        print(i)

final_href = []

for i in groups_href:
#     print(i)
    final_href.append(i[:len(i)-11]+'members')
# print(len(final_href))



In [3]:

# full data
group_name, f_name = [], []
df_name, df_contact, df_city = [], [], []


for i in final_href[1:]:
    browser.get(i)
    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)
    # scroll to bottom
    top_to_bottom(browser, 5)
    time.sleep(1.5)
    
    soup_profile = bs(browser.page_source, "html.parser")
    g_name = soup_profile.find('h1')
    group_name.append(g_name.text)
    top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")
    profile_list = []
    profile_href = []

    for i in top_elements:
        profile_list.append(i.find("a").attrs['href'])

    for i in profile_list:
        profile_href.append('https://www.facebook.com/profile.php?id='+i[i.index('user/')+5:-1])

    # get real time
    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    if kw_city:
        t_filename = f'{keyword}_{kw_city}_{t_stamp}'
    else:
        t_filename = f'{keyword}_{t_stamp}'
        
        
    name, contact, city = [], [], []
    error = []
    url = 'php?id='

    for i in profile_href:
        try:
            browser.get(i)
            temp = browser.current_url
            if url in temp:
                link = i + '&sk='
            else:
                link = temp[:temp.index('?')]
            browser.get(link+"about_contact_and_basic_info")
            soup_info = bs(browser.page_source, "html.parser")
            soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
            name.append(soup_name.find("h1").text)
            soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            contact.append(soup_info_contact.text)
            browser.get(link+"about_places")
            soup_info = bs(browser.page_source, "html.parser")
            soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            city.append(soup_info_city.text)

        except:
            l_name = len(name)
            l_cont = len(contact)
            l_city = len(city)
            print(browser.current_url)
            error.append(browser.current_url)
            if l_name > l_cont:
                contact.append('empty')
                try:
                    browser.get(link+"about_places")
                    time.sleep(1)
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_ct.text)
                except:
                    city.append('')
            elif l_cont > l_city:
                city.append('')
            else: 
                continue
    
    print_csv(t_filename, name, contact, city)
    f_name.append(t_filename)
    
timestamp = datetime.datetime.now()
t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
if kw_city:
    t_filename = f'{keyword}_{kw_city}_filelist_{t_stamp}'
else:
    t_filename = f'{keyword}_filelist_{t_stamp}'
    

df = pd.DataFrame({"그룹 이름":group_name,
                   "파일명":f_name})
df.to_csv(f"{t_filename}_utf-8.csv", encoding='UTF-8')


/groups/894440697267066/user/100002804868003/
/groups/894440697267066/user/524112894/
/groups/894440697267066/user/100006479562850/
/groups/894440697267066/user/100005051940422/
/groups/894440697267066/user/100057054392467/
/groups/894440697267066/user/100090445484491/
/groups/894440697267066/user/100003152095404/
/groups/894440697267066/user/100015367483679/
/groups/894440697267066/user/100089010903001/
/groups/894440697267066/user/100055019113980/
/groups/894440697267066/user/100071319853677/
/groups/894440697267066/user/100092641132291/
/groups/894440697267066/user/100083072365449/
/groups/894440697267066/user/100059902260191/
/groups/894440697267066/user/100022591478815/
/groups/894440697267066/user/100090145035303/
/groups/894440697267066/user/100005295972669/
/groups/894440697267066/user/100090315406964/
/groups/894440697267066/user/100080920994949/
/groups/894440697267066/user/100004813427969/
/groups/894440697267066/user/100090520733055/
/groups/894440697267066/user/10000156542

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [33]:
def print_csv(f_name, name, contact, city):
#     try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        
#         df['연락처'] = df['연락처'].str.split('\+82').str.get(1).str[0:13].fillna('없음')
#         df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        
#         int_values = []
#         for item in numbers:
#             if item.isdigit():
#                 int_values.append(int(item))
#             else:
#                 int_values.append('없음')
                
#         df['출생연도'] = int_values
#         df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
#         int_numbers = []
        
#         for i in df['연락처']:
#             if "10" in i:
#                 ii = "0" + i[1:]
#                 int_numbers.append(ii)
#             else:
#                 int_numbers.append('없음')
                
#         df['연락처'] = int_numbers
        df.to_csv(f"{f_name}_utf-8.csv", encoding='UTF-8')
#     except:
#         print(f'{len(name)} {len(contact)} {len(city)}')

In [34]:
print_csv(t_filename, name, contact, city)


In [38]:
df = pd.read_csv('음식_2023-06-23_172358_utf-8.csv')
df

,Unnamed: 0,이름,연락처,지역
0,0,이유영,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
1,1,송현숙 (노노스),연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크http://www.nono...,이전 거주지서울현재 거주지
2,2,박연지,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크yeonko._.ngInst...,이전 거주지서울현재 거주지
3,3,김나은,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보여...,이전 거주지서울현재 거주지
4,4,양지원,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
...,...,...,...,...
59,59,명세빈,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
60,60,명희철,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
61,61,모유린,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음
62,62,견희선,연락처 정보표시할 연락처 정보 없음웹사이트 및 소셜 링크표시할 링크 없음기본 정보표...,이전 거주지표시할 장소 없음


In [39]:
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        
        df['연락처'] = df['연락처'].str.split('\+82').str.get(1).str[0:13].fillna('없음')
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
                
        df['출생연도'] = int_values
        df.columns = ['번호', '이름', '연락처', '지역', '출생연도']
        int_numbers = []
        
        for i in df['연락처']:
            if "10" in i:
                ii = "0" + i[1:]
                int_numbers.append(ii)
            else:
                int_numbers.append('없음')
                
        df['연락처'] = int_numbers
        df.to_csv(f"{f_name}_utf-8.csv", encoding='UTF-8')

AttributeError: Can only use .str accessor with string values!